In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier


In [49]:
data = pd.read_csv("..\dataset\\spam_ham_dataset.csv")
X = data['text']
y = data['label_num']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_seq_length = max([len(seq) for seq in X_train_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_seq_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_seq_length, padding='post')

In [51]:
print(max_seq_length)

5916


In [52]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_seq_length))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_pad, y_train, validation_data=(X_test_pad, y_test), epochs=20, batch_size=16)

Epoch 1/20
259/259 [==============================] - 32s 116ms/step - loss: 0.6077 - accuracy: 0.7045 - val_loss: 0.5916 - val_accuracy: 0.7169
Epoch 2/20
259/259 [==============================] - 28s 108ms/step - loss: 0.5927 - accuracy: 0.7084 - val_loss: 0.5716 - val_accuracy: 0.7169
Epoch 3/20
259/259 [==============================] - 28s 107ms/step - loss: 0.5352 - accuracy: 0.7244 - val_loss: 0.4667 - val_accuracy: 0.8300
Epoch 4/20
259/259 [==============================] - 29s 110ms/step - loss: 0.3788 - accuracy: 0.8308 - val_loss: 0.3043 - val_accuracy: 0.9556
Epoch 5/20
259/259 [==============================] - 29s 111ms/step - loss: 0.2574 - accuracy: 0.9110 - val_loss: 0.2246 - val_accuracy: 0.8870
Epoch 6/20
259/259 [==============================] - 29s 112ms/step - loss: 0.1847 - accuracy: 0.9502 - val_loss: 0.1581 - val_accuracy: 0.9816
Epoch 7/20
259/259 [==============================] - 29s 111ms/step - loss: 0.1344 - accuracy: 0.9700 - val_loss: 0.1300 - val_ac

In [53]:
model.evaluate(X_test_pad, y_test)

33/33 [==============================] - 1s 28ms/step - loss: 0.1057 - accuracy: 0.9498


[0.10572800040245056, 0.9497584700584412]

In [54]:
y_pred_prob = model.predict(X_test_pad)
y_pred = np.round(y_pred_prob).astype(int)
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

print("confusion matrix:")
print(conf_matrix)
print("True Positives (TP) :", tp)
print("False Positives (TN) :", tn)

33/33 [==============================] - 1s 28ms/step
confusion matrix:
[[740   2]
 [ 50 243]]
True Positives (TP) : 243
False Positives (TN) : 740


In [55]:
model.save("..\\models\\mail.keras")

In [62]:
import pickle

# saving tokenizer
with open('..\\models\\mailtoken.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [63]:
with open('..\\models\\mailtoken.pickle', 'rb') as handle:
    t = pickle.load(handle)

In [64]:
X_test_seq = t.texts_to_sequences(X_test)
maxlen1=5916
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen1, padding='post')

In [65]:
from keras.models import load_model
model1 = load_model("..\\models\\mail.keras")
model1.evaluate(X_test_pad, y_test)
model1.predict(X_test_pad)

33/33 [==============================] - 1s 32ms/step


array([[5.5028366e-05],
       [1.0000000e+00],
       [4.4215403e-07],
       ...,
       [9.9547392e-01],
       [1.3544263e-01],
       [1.1087752e-05]], dtype=float32)

In [66]:
text=["hello this is a text for guyfiyudyx testing your power","time to die"]
df1=pd.DataFrame(text,columns=["text"])
x=df1["text"]
X_test_seq = t.texts_to_sequences(x)
maxlen1=5916
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen1, padding='post')
model1.predict(X_test_pad)

1/1 [==============================] - 0s 28ms/step


array([[0.397237  ],
       [0.32604584]], dtype=float32)